In [2]:
import numpy as np # linear algebra
import pandas as pd 

# visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot

import gc

from catboost import Pool
import catboost as ctb
from catboost import *
import lightgbm as lgb
import xgboost as xgb


from sklearn import metrics
from time import time

import os
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix,mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [3]:
df_train=pd.read_feather('C:/Users/sony6/OneDrive/바탕 화면/AMEX_FTR/train_data.ftr')
df_test=pd.read_feather('C:/Users/sony6/OneDrive/바탕 화면/AMEX_FTR/test_data.ftr')

In [4]:
train_df=df_train.groupby(['customer_ID']).tail(1)
test_df=df_test.groupby(['customer_ID']).tail(1)

In [5]:
num_cols = df_train.select_dtypes([np.int64,np.float16]).columns
cat_cols=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [6]:
lab_enc = LabelEncoder()
for cat_feat in cat_cols:
    train_df[cat_feat] = lab_enc.fit_transform(train_df[cat_feat])
    test_df[cat_feat] = lab_enc.transform(test_df[cat_feat])

C:\Users\sony6\AppData\Local\Temp\ipykernel_8200\2016194234.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[cat_feat] = lab_enc.fit_transform(train_df[cat_feat])
C:\Users\sony6\AppData\Local\Temp\ipykernel_8200\2016194234.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[cat_feat] = lab_enc.transform(test_df[cat_feat])


In [7]:
X,Y=train_df.drop(['customer_ID','target','S_2'],axis=1),train_df['target']


#making sure we have same columns in test data as in train
col=[c for c in X.columns]
test_x=test_df[col]

In [8]:
del test_df,df_train,df_test,train_df
gc.collect()

327

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)


categorical_features_indices=[]
for c in cat_cols:
    a=X_train.columns.get_loc(c)
    categorical_features_indices.append(a)

In [10]:
del X,Y
gc.collect()

0

In [11]:
print(f"""
X_train shape: {X_train.shape}
X_test shape: {X_test.shape}
y_train shape: {y_train.shape}
y_test shape: {y_test.shape}
""")


X_train shape: (367130, 188)
X_test shape: (91783, 188)
y_train shape: (367130,)
y_test shape: (91783,)



In [12]:
learning_rate=np.linspace(0.01,0.1,10)
max_depth=np.arange(2, 18, 2)
colsample_bylevel=np.arange(0.3, 0.8, 0.1)
iterations=np.arange(50, 1000, 50)
l2_leaf_reg=np.arange(0,10)
bagging_temperature=np.arange(0,100,10)
n_estimators=np.arange(50,500,50)

In [13]:
xgb_cat_params = {
    'learning_rate':    hp.choice('learning_rate',    learning_rate),
    'max_depth':        hp.choice('max_depth',         max_depth),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bylevel),
    'n_estimators':     hp.choice('n_estimators',    n_estimators),
    'loss_function':       'logloss',
    'nan_mode':'Min',
    'task_type':'GPU'
}
xgb_fit_params = {
    'eval_metric': 'logloss',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['cls_params'] = xgb_cat_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# LightGBM parameters
lgb_cat_params = {
    'learning_rate':    hp.choice('learning_rate',    learning_rate),
    'max_depth':        hp.choice('max_depth',        max_depth),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bylevel),
    'n_estimators':     hp.choice('n_estimators',    n_estimators),
    'loss_function':       'CrossEntropy',
    'nan_mode':'Min'
}
lgb_fit_params = {
    'eval_metric': 'CrossEntropy',
    'early_stopping_rounds': 10,
    'verbose': False
}
lgb_para = dict()
lgb_para['cls_params'] = lgb_cat_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# Catboost parameters
ctb_cat_params = {
    'learning_rate':     hp.choice('learning_rate',    learning_rate),
    'max_depth':         hp.choice('max_depth',         max_depth),
    'iterations':       hp.choice('iterations',            iterations),
    'loss_function':       'CrossEntropy',
    'nan_mode':'Min',
    'task_type':'GPU'
    
}
ctb_fit_params = {
    'early_stopping_rounds': 5,
    'verbose': False,
    'cat_features': categorical_features_indices
}
ctb_para = dict()
ctb_para['cls_params'] = ctb_cat_params
ctb_para['fit_params'] = ctb_fit_params
ctb_para['loss_func' ] = lambda y, pred: mean_absolute_error(y, pred)

In [14]:
class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            print('entering fmin')
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result

    def ctb_cls(self, para):
        cls = ctb.CatBoostClassifier(**para['cls_params'])
        print('ctb initialized')
        return self.train_cls(cls, para)
    
    def xgb_cls(self, para):
        cls = xgb.XGBClassifier(**para['cls_params'])
        print('ctb initialized')
        return self.train_cls(cls, para)
    
    def lgb_cls(self, para):
        cls = lgb.LGBMClassifier(**para['cls_params'])
        print('ctb initialized')
        return self.train_cls(cls, para)

    def train_cls(self, cls, para):
        print('fitting model')
        cls.fit(self.x_train, self.y_train,
                eval_set=[(self.x_test, self.y_test)],
                **para['fit_params'])
        print('model fitted')
        pred = cls.predict(self.x_test)
        #loss = para['loss_func'](self.y_test, pred)
        f1=sklearn.metrics.f1_score(self.y_test,pred)
        f1=f1*(-1)
        print(f1)
        return {'loss': f1, 'status': STATUS_OK}

In [15]:
obj = HPOpt(X_train, X_test, y_train, y_test)
ctb_opt = obj.process(fn_name='ctb_cls', space=ctb_para, trials=Trials(), algo=tpe.suggest, max_evals=2)

entering fmin
ctb initialized                                      
fitting model                                        
model fitted                                         
-0.8074887901906199                                  
ctb initialized                                                                 
fitting model                                                                   
model fitted                                                                    
-0.7441130298273154                                                             
100%|██████████| 2/2 [00:46<00:00, 23.08s/trial, best loss: -0.8074887901906199]


In [16]:
ctb_opt

{'iterations': 14, 'learning_rate': 7, 'max_depth': 2}

In [17]:
best_param_ctb={}
best_param_ctb['learning_rate']=learning_rate[ctb_opt['learning_rate']]
best_param_ctb['iterations']=iterations[ctb_opt['iterations']]
best_param_ctb['max_depth']=max_depth[ctb_opt['max_depth']]

In [18]:
best_param_ctb

{'learning_rate': 0.08, 'iterations': 750, 'max_depth': 6}

In [19]:
lgb_opt = obj.process(fn_name='lgb_cls', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=2)

entering fmin
ctb initialized                                      
fitting model                                        
  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

c:\Users\sony6\.conda\envs\tf\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

c:\Users\sony6\.conda\envs\tf\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] Unknown parameter: nan_mode     
model fitted                                         
-0.8059954631219657                                  
ctb initialized                                                                 
fitting model                                                                   
 50%|█████     | 1/2 [00:04<00:04,  4.81s/trial, best loss: -0.8059954631219657]

c:\Users\sony6\.conda\envs\tf\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

c:\Users\sony6\.conda\envs\tf\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[LightGBM] [Warning] Unknown parameter: loss_function                           
[LightGBM] [Warning] Unknown parameter: nan_mode                                
model fitted                                                                    
-0.8036744738628648                                                             
100%|██████████| 2/2 [00:09<00:00,  4.51s/trial, best loss: -0.8059954631219657]


In [20]:
lgb_opt

{'colsample_bytree': 1, 'learning_rate': 5, 'max_depth': 7, 'n_estimators': 8}

In [21]:
best_param_lgb={}
best_param_lgb['learning_rate']=learning_rate[lgb_opt['learning_rate']]
best_param_lgb['colsample_bytree']=colsample_bylevel[lgb_opt['colsample_bytree']]
best_param_lgb['max_depth']=max_depth[lgb_opt['max_depth']]
best_param_lgb['n_estimators']=n_estimators[lgb_opt['n_estimators']]

In [22]:
best_param_lgb

{'learning_rate': 0.06000000000000001,
 'colsample_bytree': 0.4,
 'max_depth': 16,
 'n_estimators': 450}

In [23]:
xgb_opt = obj.process(fn_name='xgb_cls', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=2)

entering fmin
ctb initialized                                      
fitting model                                        
  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

c:\Users\sony6\.conda\envs\tf\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

c:\Users\sony6\.conda\envs\tf\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

c:\Users\sony6\.conda\envs\tf\lib\site-packages\xgboost\core.py:160: UserWarning: [10:13:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "loss_function", "nan_mode", "task_type" } are not used.

  warnings.warn(smsg, UserWarning)



model fitted                                         
-0.8019827256094966                                  
ctb initialized                                                                 
fitting model                                                                   
 50%|█████     | 1/2 [00:44<00:44, 44.46s/trial, best loss: -0.8019827256094966]

c:\Users\sony6\.conda\envs\tf\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

c:\Users\sony6\.conda\envs\tf\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

c:\Users\sony6\.conda\envs\tf\lib\site-packages\xgboost\core.py:160: UserWarning: [10:14:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "loss_function", "nan_mode", "task_type" } are not used.

  warnings.warn(smsg, UserWarning)



model fitted                                                                    
-0.7920955125566077                                                             
100%|██████████| 2/2 [01:43<00:00, 51.73s/trial, best loss: -0.8019827256094966]


In [24]:
xgb_opt

{'colsample_bytree': 1, 'learning_rate': 4, 'max_depth': 5, 'n_estimators': 1}

In [25]:
best_param_xgb={}
best_param_xgb['learning_rate']=learning_rate[xgb_opt['learning_rate']]
best_param_xgb['colsample_bytree']=colsample_bylevel[xgb_opt['colsample_bytree']]
best_param_xgb['max_depth']=max_depth[xgb_opt['max_depth']]
best_param_xgb['n_estimators']=n_estimators[xgb_opt['n_estimators']]

In [26]:
best_param_xgb

{'learning_rate': 0.05000000000000001,
 'colsample_bytree': 0.4,
 'max_depth': 12,
 'n_estimators': 100}

In [27]:
model_lgb=lgb.LGBMClassifier(n_estimators=best_param_lgb['n_estimators'], 
                             depth=best_param_lgb['max_depth'],
                             learning_rate=best_param_lgb['learning_rate'],
                             colsample_bytree=best_param_lgb['colsample_bytree'],
                            loss_function='logloss',
                             nan_mode='Min',
                            #task_type='GPU',
                             
                            random_seed=42
                            )

In [28]:
model_lgb.fit(X_train,y_train, eval_set=None)

[LightGBM] [Warning] Unknown parameter: loss_function
[LightGBM] [Warning] Unknown parameter: depth
[LightGBM] [Warning] Unknown parameter: nan_mode


LGBMClassifier(colsample_bytree=0.4, depth=16,
               learning_rate=0.06000000000000001, loss_function='logloss',
               n_estimators=450, nan_mode='Min', random_seed=42)

In [29]:
model_xgb=xgb.XGBClassifier(n_estimators=best_param_xgb['n_estimators'], 
                             depth=best_param_xgb['max_depth'],
                             learning_rate=best_param_xgb['learning_rate'],
                             colsample_bytree=best_param_xgb['colsample_bytree'],
                            loss_function='logloss',
                             nan_mode='Min',
                            task_type='GPU',
                             
                            random_seed=42
                            )

In [30]:
model_xgb.fit(X_train,y_train, eval_set=None)

c:\Users\sony6\.conda\envs\tf\lib\site-packages\xgboost\core.py:160: UserWarning: [10:15:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "depth", "loss_function", "nan_mode", "random_seed", "task_type" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, depth=12, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05000000000000001,
              loss_function='logloss', max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, ...)

In [31]:
model_ctb=ctb.CatBoostClassifier(iterations=best_param_ctb['iterations'], 
                             depth=best_param_ctb['max_depth'],
                             learning_rate=best_param_ctb['learning_rate'],
                            loss_function='CrossEntropy',
                             nan_mode='Min',
                            task_type='GPU',
                            random_seed=42
                            )

In [32]:
model_ctb.fit(X_train,y_train,cat_features=categorical_features_indices, eval_set=None, plot=False)

0:	learn: 0.5991115	total: 32.7ms	remaining: 24.5s
1:	learn: 0.5277448	total: 65.4ms	remaining: 24.4s
2:	learn: 0.4723635	total: 103ms	remaining: 25.7s
3:	learn: 0.4307177	total: 135ms	remaining: 25.2s
4:	learn: 0.3984856	total: 166ms	remaining: 24.8s
5:	learn: 0.3744123	total: 199ms	remaining: 24.7s
6:	learn: 0.3527685	total: 231ms	remaining: 24.5s
7:	learn: 0.3364935	total: 263ms	remaining: 24.4s
8:	learn: 0.3217600	total: 296ms	remaining: 24.3s
9:	learn: 0.3109254	total: 328ms	remaining: 24.2s
10:	learn: 0.3030414	total: 359ms	remaining: 24.1s
11:	learn: 0.2942944	total: 390ms	remaining: 24s
12:	learn: 0.2884861	total: 422ms	remaining: 23.9s
13:	learn: 0.2824370	total: 454ms	remaining: 23.9s
14:	learn: 0.2782079	total: 486ms	remaining: 23.8s
15:	learn: 0.2748564	total: 517ms	remaining: 23.7s
16:	learn: 0.2702530	total: 548ms	remaining: 23.6s
17:	learn: 0.2675527	total: 580ms	remaining: 23.6s
18:	learn: 0.2640885	total: 612ms	remaining: 23.5s
19:	learn: 0.2620482	total: 644ms	remaini

In [33]:
#Catboost
y_pred_ctb=model_ctb.predict(X_test)

In [34]:
#XGBoost
y_pred_xgb=model_xgb.predict(X_test)

In [35]:
#LGBM
y_pred_lgb=model_lgb.predict(X_test)

In [36]:
print('F1-score of Catboost: {:.2f}'.format(f1_score(y_test, y_pred_ctb)))
print('F1-score of XGBoost: {:.2f}'.format(f1_score(y_test, y_pred_xgb)))
print('F1-score of LGBM: {:.2f}'.format(f1_score(y_test, y_pred_lgb)))

F1-score of Catboost: 0.81
F1-score of XGBoost: 0.80
F1-score of LGBM: 0.80


In [37]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [38]:
y_test1 = pd.DataFrame(y_test, columns=['target'])
y_pred1 = pd.DataFrame(y_pred_ctb, columns=['prediction'])

In [39]:
print(amex_metric(y_test1, y_pred1))

-0.0309543478051363
